# Test model example

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path

import matplotlib.pyplot as plt
import neuroplotlib as nplt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import shutil

import model
import evaluator
import plotting
import utils

# %matplotlib notebook
%matplotlib notebook

In [ ]:
probe_type = "planar"
model_name = "hay_ais" #'hallermann' # "hay" hay_ais
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".").absolute()
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")
    compiled_folder = Path("./x86_64")
    if compiled_folder.is_dir():
        shutil.rmtree(compiled_folder)
    shutil.copytree(model_folder / "x86_64", compiled_folder)

In [ ]:
protocols_file = "../data/hay_ecode_probe_planar/efeatures/protocols_BPO.json"
features_file = "../data/hay_ecode_probe_planar/efeatures/features_BPO.json"

In [ ]:
ais_recordings = utils.get_ais_extra_recordings()
ais_recordings

In [ ]:
cell = model.create(model_name=model_name, release=True)

probe = model.define_electrode(probe_type=probe_type)
# probe=None

param_names = [param.name for param in cell.params.values() if not param.frozen]

fitness_protocols = evaluator.define_protocols(
    model_name,
    feature_set="soma",
    feature_file=features_file,#f"{model_name}_model/features_list.json",
    protocols_file=protocols_file,
    electrode=probe,
    protocols_with_lfp=["firepattern_200"],
    extra_recordings=dict(firepattern_200=ais_recordings)  #["Step1", "bAP", "BAC", "EPSP", "CaBurst" ]
)

fitness_calculator = ephys.objectivescalculators.ObjectivesCalculator([])

if model_name == 'hallermann':
    sim = ephys.simulators.LFPySimulator(cell, cvode_active=False, electrode=probe)
else:
    sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

eva = ephys.evaluators.CellEvaluator(                    
        cell_model=cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)

In [ ]:
fitness_protocols

In [ ]:
firepattern_200 = fitness_protocols["firepattern_200"]
# print(eva)

In [ ]:
firepattern_200.recordings

In [ ]:
t_start = time.time()
responses = eva.run_protocols(protocols=fitness_protocols.values(), param_values={})
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
# Sanity check for soma diam. Soma area pi*L*diam != soma h.area(0.5)
# pi*23.169*500 = 36393.7800955 != 1131.3891 

# But soma h.area(0.5) from BPO == L5PCtemplate[0].soma[0] area(0.5) is true

# oc>access L5PCtemplate[0].soma[0]
# oc>area(0.5)
# 	1131.3891 

#                         ^
# oc>access L5PCtemplate[0].axon[0]
# oc>area(0.5)
# 	94.24778 
# oc>access L5PCtemplate[0].axon[1]
# oc>area(0.5)
# 	94.24778 
# oc>


# L5PCtemplate[0].axon[1] { nseg=1  L=30  Ra=100
# 	L5PCtemplate[0].axon[0] connect L5PCtemplate[0].axon[1] (0), 1
# 	/* First segment only */
# 	insert morphology { diam=1}
# 	insert capacitance { cm=1}
# 	insert pas { g_pas=3.25e-05 e_pas=-90}
# }


# oc>secname()
# L5PCtemplate[0].axon[0]
# oc>psection()
# L5PCtemplate[0].axon[0] { nseg=1  L=30  Ra=100
# 	L5PCtemplate[0].soma[0] connect L5PCtemplate[0].axon[0] (0), 0.5
# 	/* First segment only */
# 	insert morphology { diam=1}
# 	insert capacitance { cm=1}
# 	insert pas { g_pas=3.25e-05 e_pas=-90}
# }


In [ ]:
plt.figure()
for resp_name, response in sorted(responses.items()):
    if '.v' in resp_name and 'Step1' in resp_name:
        plt.plot(response['time'], response['voltage'], label=resp_name, lw=3)
plt.legend(fontsize=20)
plt.xlabel("Time (ms)", fontsize=20)
plt.ylabel("Membrane potential (mV)", fontsize=20)
plt.xlim([700,710])

In [ ]:
plt.figure()
for resp_name, response in sorted(responses.items()):
    if '.v' in resp_name:
        plt.plot(response['time'], response['voltage'], label=resp_name, lw=3)

In [ ]:
cell.freeze(param_dict={})
cell.instantiate(sim=sim)

In [ ]:
nplt.plot_detailed_neuron(cell.LFPyCell, plane="xy", color_ais="g", color_hillock="k", color_myelin="m")

In [ ]:
fig, ax = plt.subplots(2, figsize=(8,8))
for resp_name, response in sorted(responses.items()):
    if 'i_membrane' in resp_name and 'Step1' in resp_name:
        ax[0].plot(response['time'], response['voltage'], label=resp_name.split('.i_membrane')[0])
        
    if '.v' in resp_name and 'Step1' in resp_name:
        ax[1].plot(response['time'], response['voltage'], label=resp_name)
        
ax[0].legend() #bbox_to_anchor=(0.5,0.5)
ax[0].set_xlim([701,706])
# ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('i_membrane (mA/cm$^2$)')
ax[0].grid()

ax[1].set_xlim([701,706])
ax[1].set_xlabel('Time (ms)')
ax[1].set_ylabel('Vm(mV)')
ax[1].legend() #bbox_to_anchor=(0.5,0.5)
ax[1].grid()

plt.tight_layout() 



In [ ]:
fig, ax = plt.subplots(2, figsize=(8,8))
for resp_name, response in sorted(responses.items()):
    if 'i_membrane' in resp_name and 'Step1' in resp_name and 'soma' in resp_name:
        ax[0].plot(response['time'], response['voltage']*1131.3890593405758*0.01, label=resp_name.split('.i_membrane')[0])
    elif 'i_membrane' in resp_name and 'Step1' in resp_name and 'hillock' in resp_name:
        ax[0].plot(response['time'], response['voltage']*15.28114274318315*0.01, label=resp_name.split('.i_membrane')[0])
    elif 'i_membrane' in resp_name and 'Step1' in resp_name and 'ais' in resp_name:
        ax[0].plot(response['time'], response['voltage']*14.128661776062582*0.01, label=resp_name.split('.i_membrane')[0])
        
    if '.v' in resp_name and 'Step1' in resp_name:
        ax[1].plot(response['time'], response['voltage'], label=resp_name)
        
ax[0].legend() #bbox_to_anchor=(0.5,0.5)
ax[0].set_xlim([701,706])
# ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('i_membrane (nA)')
ax[0].grid()

ax[1].set_xlim([701,706])
ax[1].set_xlabel('Time (ms)')
ax[1].set_ylabel('Vm(mV)')
ax[1].legend() #bbox_to_anchor=(0.5,0.5)
ax[1].grid()

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(2, figsize=(8,8))
for resp_name, response in sorted(responses.items()):
    if 'i_membrane' in resp_name and 'Step1' in resp_name and 'soma' in resp_name:
        ax[0].plot(response['time'], response['voltage']*1131.3890593405758*0.01, label=resp_name.split('.i_membrane')[0])      
    if '.v' in resp_name and 'Step1' in resp_name:
        ax[1].plot(response['time'], response['voltage'], label=resp_name)
        
ax[0].legend() #bbox_to_anchor=(0.5,0.5)
ax[0].set_xlim([701,706])
# ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('i_membrane (nA)')
ax[0].grid()

ax[1].set_xlim([701,706])
ax[1].set_xlabel('Time (ms)')
ax[1].set_ylabel('Vm(mV)')
ax[1].legend() #bbox_to_anchor=(0.5,0.5)
ax[1].grid()

plt.tight_layout() 


# Soma area =  1131.3890593405758
# Hillock area(0.5) =  15.28114274318315
# AIS area(0.5) =  14.128661776062582

In [ ]:
fig, ax = plt.subplots(2, figsize=(8,8))
for resp_name, response in sorted(responses.items()):
    if 'i_membrane' in resp_name and 'Step1' in resp_name and 'hillock' in resp_name:
        ax[0].plot(response['time'], response['voltage']*15.28114274318315*0.01, label=resp_name.split('.i_membrane')[0])
        
    if '.v' in resp_name and 'Step1' in resp_name:
        ax[1].plot(response['time'], response['voltage'], label=resp_name)
        
ax[0].legend() #bbox_to_anchor=(0.5,0.5)
ax[0].set_xlim([701,706])
# ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('i_membrane (nA)')
ax[0].grid()

ax[1].set_xlim([701,706])
ax[1].set_xlabel('Time (ms)')
ax[1].set_ylabel('Vm(mV)')
ax[1].legend() #bbox_to_anchor=(0.5,0.5)
ax[1].grid()

plt.tight_layout() 

In [ ]:
fig, ax = plt.subplots(2, figsize=(8,8))
for resp_name, response in sorted(responses.items()):
    if 'i_membrane' in resp_name and 'Step1' in resp_name and 'ais' in resp_name:
        ax[0].plot(response['time'], response['voltage']*14.128661776062582*0.01, label=resp_name.split('.i_membrane')[0])
        
    if '.v' in resp_name and 'Step1' in resp_name:
        ax[1].plot(response['time'], response['voltage'], label=resp_name)
        
ax[0].legend() #bbox_to_anchor=(0.5,0.5)
ax[0].set_xlim([701,706])
# ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('i_membrane (nA)')
ax[0].grid()

ax[1].set_xlim([701,706])
ax[1].set_xlabel('Time (ms)')
ax[1].set_ylabel('Vm(mV)')
ax[1].legend() #bbox_to_anchor=(0.5,0.5)
ax[1].grid()

plt.tight_layout() 


In [ ]:
import numpy as np
time_array = np.array(responses['Step1.soma_mid_imembrane.i_membrane']['time'])
itotal  = np.zeros(time_array.shape)


fig, ax = plt.subplots(2, figsize=(6,6))
for resp_name, response in sorted(responses.items()):
    if 'Step1.soma_i' in resp_name:
#         print(resp_name)
        itotal = itotal+np.array(response['voltage'])
        
    if 'Step1.soma_mid_imembrane.i_membrane' in resp_name: # and 'Step1' in resp_name:
        ax[1].plot(response['time'], response['voltage'], label=resp_name)
#         ax[0].plot(response['time'], response['voltage'], label=resp_name)
        

ax[0].plot(time_array, itotal, label='Sum_membrane')
ax[0].legend(bbox_to_anchor=(0.5,0.5))
ax[0].set_xlim([701,706])
# ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('i_membrane (mA/cm$^2$)')
ax[0].grid()

ax[1].set_xlim([701,706])
ax[1].set_xlabel('Time (ms)')
# ax[1].set_ylabel('Vm(mV)')
ax[0].set_ylabel('i_membrane (mA/cm$^2$)')
ax[1].legend(bbox_to_anchor=(0.5,0.5))
ax[1].grid()

plt.tight_layout() 


In [ ]:
import numpy as np
time_array = np.array(responses['Step1.hillock_mid_imembrane.i_membrane']['time'])
itotal  = np.zeros(time_array.shape)


fig, ax = plt.subplots(2, figsize=(6,6))
for resp_name, response in sorted(responses.items()):
    if 'Step1.hil_i' in resp_name:
        print(resp_name)
        itotal = itotal+np.array(response['voltage'])
        
    if 'Step1.hillock_mid_imembrane.i_membrane' in resp_name: # and 'Step1' in resp_name:
        ax[1].plot(response['time'], response['voltage'], label=resp_name)
#         ax[0].plot(response['time'], response['voltage'], label=resp_name)
        

ax[0].plot(time_array, itotal, label='Sum_membrane_hillock')
ax[0].legend(bbox_to_anchor=(0.5,0.5))
ax[0].set_xlim([701,706])
# ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('i_membrane (mA/cm$^2$)')
ax[0].grid()

ax[1].set_xlim([701,706])
ax[1].set_xlabel('Time (ms)')
# ax[1].set_ylabel('Vm(mV)')
ax[0].set_ylabel('i_membrane (mA/cm$^2$)')
ax[1].legend(bbox_to_anchor=(0.5,0.5))
ax[1].grid()

plt.tight_layout() 

In [ ]:
import numpy as np
time_array = np.array(responses['Step1.hillock_mid_imembrane.i_membrane']['time'])
itotal  = np.zeros(time_array.shape)


fig, ax = plt.subplots(2, figsize=(6,6))
for resp_name, response in sorted(responses.items()):
    if 'Step1.hil_i' in resp_name:
        print(resp_name)
        itotal = itotal+np.array(response['voltage'])
        ax[0].plot(response['time'], response['voltage'], label=resp_name)

        
    if 'Step1.hillock_mid_imembrane.i_membrane' in resp_name: # and 'Step1' in resp_name:
        ax[1].plot(response['time'], response['voltage'], label=resp_name)
#         ax[0].plot(response['time'], response['voltage'], label=resp_name)
        

ax[0].plot(time_array, itotal, label='Sum_membrane_hillock')
ax[0].legend(bbox_to_anchor=(1,0.5))
ax[0].set_xlim([701,706])
# ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('i_membrane (mA/cm$^2$)')
ax[0].grid()

ax[1].set_xlim([701,706])
ax[1].set_xlabel('Time (ms)')
# ax[1].set_ylabel('Vm(mV)')
ax[0].set_ylabel('i_membrane (mA/cm$^2$)')
ax[1].legend(bbox_to_anchor=(0.5,0.5))
ax[1].grid()

plt.tight_layout() 

In [ ]:
#### import numpy as np
time_array = np.array(responses['Step1.hillock_mid_imembrane.i_membrane']['time'])

fig, ax = plt.subplots(2, figsize=(6,6))
for resp_name, response in sorted(responses.items()):
    if 'Step1.hil_current' in resp_name:
        print(resp_name)
        ax[0].plot(response['time'], response['voltage'], label=resp_name)

        
    if 'Step1.hillock_mid_imembrane.i_membrane' in resp_name: # and 'Step1' in resp_name:
        ax[1].plot(response['time'], response['voltage'], label=resp_name)        

ax[0].legend(bbox_to_anchor=(1,0.5))
ax[0].set_xlim([701,706])
# ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('i_membrane (mA/cm$^2$)')
ax[0].grid()

ax[1].set_xlim([701,706])
ax[1].set_xlabel('Time (ms)')
ax[0].set_ylabel('i_membrane (mA/cm$^2$)')
ax[1].legend(bbox_to_anchor=(0.5,0.5))
ax[1].grid()

plt.tight_layout() 

In [ ]:

time_array = np.array(responses['Step1.hillock_mid_imembrane.i_membrane']['time'])

plt.figure()
for resp_name, response in sorted(responses.items()):
    if 'Step1.hil_currentNaTg.ina_NaTg' in resp_name:
        print(resp_name)
        plt.plot(response['time'], response['voltage'], label=resp_name)
    
plt.legend(bbox_to_anchor=(0.5,0.5))
plt.xlim([701,706])
# plt.set_xlabel('Time (ms)')
plt.ylabel('i_membrane (mA/cm$^2$)')
plt.grid()

In [ ]:
time_array = np.array(responses['Step1.hillock_mid_imembrane.i_membrane']['time'])

plt.figure()
for resp_name, response in sorted(responses.items()):
    if 'Step1.hil_currentNaTg.ina_NaTg' in resp_name:
        print(resp_name)
        plt.plot(response['time'], response['voltage'], label=resp_name)
    
plt.legend(bbox_to_anchor=(0.5,0.5))
plt.xlim([701,706])
# plt.set_xlabel('Time (ms)')
plt.ylabel('i_membrane (mA/cm$^2$)')
plt.grid()

In [ ]:
import numpy as np
time_array = np.array(responses['Step1.ais_mid_imembrane.i_membrane']['time'])
itotal  = np.zeros(time_array.shape)


fig, ax = plt.subplots(2, figsize=(6,6))
for resp_name, response in sorted(responses.items()):
    if 'Step1.ais_i' in resp_name:
        print(resp_name)
        itotal = itotal+np.array(response['voltage'])
        
    if 'Step1.ais_mid_imembrane.i_membrane' in resp_name: # and 'Step1' in resp_name:
        ax[1].plot(response['time'], response['voltage'], label=resp_name)
#         ax[0].plot(response['time'], response['voltage'], label=resp_name)
        

ax[0].plot(time_array, itotal, label='Sum_membrane_ais')
ax[0].legend(bbox_to_anchor=(0.5,0.5))
ax[0].set_xlim([701,706])
# ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('i_membrane (mA/cm$^2$)')
ax[0].grid()

ax[1].set_xlim([701,706])
ax[1].set_xlabel('Time (ms)')
# ax[1].set_ylabel('Vm(mV)')
ax[0].set_ylabel('i_membrane (mA/cm$^2$)')
ax[1].legend(bbox_to_anchor=(0.5,0.5))
ax[1].grid()

plt.tight_layout() 

In [ ]:
import neurom # https://github.com/BlueBrain/NeuroM
import neurom.viewer

In [ ]:
neurom.viewer.draw(neurom.load_neuron('hay_ais_model/morphology.asc'))